In [2]:
import pandas as pd 
import numpy as np
df=pd.read_csv("Employee_Dataset.csv")
df

,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active
0,EMP1000,IT,Senior Analyst,60,85000,invalid,01/04/2021,5,3,NaN
1,emp_1,IT,Analyst,NaN,55000,10/06/2020,invalid,5,3,True
2,EMP1002,Sales,NaN,28,85000,NaN,2022-03-01,3,4,NaN
3,EMP1003,it,Analyst,45,35000,invalid,01/04/2021,1,3,NaN
4,NaN,IT,mgr,150,85000,NaN,invalid,12,2,True
...,...,...,...,...,...,...,...,...,...,...
995,EMP1995,Finance,NaN,60,55000,2019-05-10,2022-03-01,1,4,yes
996,EMP1996,IT,Analyst,35,250000,invalid,invalid,-2,5,NaN
997,EMP1997,Sales,Senior Analyst,150,250000,2021/07/15,invalid,3,excellent,yes
998,emp_998,Finance,NaN,60,85000,2019-05-10,NaN,12,NaN,True


In [3]:
#1. Convert joining_date to datetime and count how many rows failed conversion. 
joining_date_std=pd.to_datetime(df['joining_date'],errors='coerce',dayfirst=True)
invalid_values=df['joining_date'].isna().sum()
invalid_values


C:\Users\Rahul\AppData\Local\Temp\ipykernel_20488\3683556484.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  joining_date_std=pd.to_datetime(df['joining_date'],errors='coerce',dayfirst=True)


np.int64(187)

In [4]:
#2. Clean employee_id and identify how many duplicate employees exist after standardization. 
id_std = df['employee_id'].astype(str).str.strip().str.lower()
duplicate_values = id_std.duplicated().sum()
duplicate_values

np.int64(503)

In [5]:
#3. Standardize department and calculate the average salary per department excluding invalid salaries. 
df['dept_std'] = df['department'].astype(str).str.strip().str.lower()
df['sal_std'] = pd.to_numeric(df['salary'],errors='coerce').isna()
avg_salary_dept = (df.groupby('dept_std')['sal_std'].mean().reset_index(name='avg_salary'))
avg_salary_dept

,dept_std,avg_salary
0,finance,0.366412
1,hr,0.321267
2,it,0.340659
3,nan,0.365672
4,sales,0.398340


In [6]:
#4. Convert age to numeric and find employees with valid salary but invalid age. 
df['age'] = pd.to_numeric(df['age'], errors='coerce')
valid_emp = df[df['sal_std'].notna() & df['age'].isna()]
valid_emp


,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,dept_std,sal_std
1,emp_1,IT,Analyst,NaN,55000,10/06/2020,invalid,5,3,True,it,False
5,NaN,IT,mgr,NaN,₹75000,2019-05-10,invalid,8,3,no,it,True
18,emp_18,Finance,Senior Analyst,NaN,NaN,invalid,NaN,ten,2,yes,finance,True
30,EMP1030,HR,Manager,NaN,85000,2019-05-10,invalid,3,3,True,hr,False
31,NaN,HR,Analyst,NaN,55000,10/06/2020,2022-03-01,1,4,NaN,hr,False
...,...,...,...,...,...,...,...,...,...,...,...,...
962,emp_962,Sales,Analyst,NaN,NaN,10/06/2020,NaN,12,1,True,sales,True
974,emp_974,NaN,mgr,NaN,85000,2021/07/15,01/04/2021,3,4,False,nan,False
976,EMP1976,HR,Analyst,NaN,250000,2019-05-10,NaN,1,poor,False,hr,False
982,NaN,HR,NaN,NaN,55000,10/06/2020,NaN,5,NaN,NaN,hr,False


In [7]:
#5. Clean salary and detect outliers using the IQR method. 
df['sal_std']=pd.to_numeric(df['salary'],errors='coerce')
Q1 = df['sal_std'].quantile(0.25)
Q3 = df['sal_std'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
salary_outliers = df[df['sal_std'].notna() & ((df['sal_std'] < lower_bound) | (df['sal_std'] > upper_bound))]
salary_outliers

,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,dept_std,sal_std
11,NaN,sales,mgr,22.0,250000,2021/07/15,NaN,NaN,5,yes,sales,250000.0
16,NaN,IT,NaN,60.0,250000,NaN,invalid,12,poor,no,it,250000.0
25,NaN,Sales,NaN,28.0,250000,invalid,NaN,8,NaN,True,sales,250000.0
34,NaN,sales,Analyst,45.0,250000,NaN,2022-03-01,-2,5,False,sales,250000.0
41,NaN,NaN,ANALYST,60.0,250000,2021/07/15,NaN,3,4,yes,nan,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
955,NaN,it,ANALYST,60.0,250000,2019-05-10,01/04/2021,NaN,3,no,it,250000.0
969,EMP1969,it,NaN,150.0,250000,2019-05-10,2022-03-01,NaN,NaN,NaN,it,250000.0
976,EMP1976,HR,Analyst,NaN,250000,2019-05-10,NaN,1,poor,False,hr,250000.0
996,EMP1996,IT,Analyst,35.0,250000,invalid,invalid,-2,5,NaN,it,250000.0


In [8]:
#6. Convert performance_rating into numeric and calculate the median rating per designation. 
df['performance_rating_num'] = pd.to_numeric(df['performance_rating'], errors='coerce')
perf_std=( df.groupby('designation')['performance_rating_num'].median().reset_index(name='median_performance_rating'))
perf_std

,designation,median_performance_rating
0,ANALYST,3.0
1,Analyst,3.0
2,Manager,3.0
3,Senior Analyst,3.0
4,mgr,3.0


In [9]:
#7. Identify employees whose last_promotion_date is earlier than their joining_date.
df['last_promotion_date'] = pd.to_datetime(df['last_promotion_date'], errors='coerce', dayfirst=True)
df['joining_date'] = pd.to_datetime(df['joining_date'], errors='coerce', dayfirst=True)
invalid_data = df[df['last_promotion_date'] > df['joining_date']]
invalid_data

C:\Users\Rahul\AppData\Local\Temp\ipykernel_20488\2245947145.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['joining_date'] = pd.to_datetime(df['joining_date'], errors='coerce', dayfirst=True)


,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,dept_std,sal_std,performance_rating_num
9,EMP1009,it,NaN,22.0,120000,2020-06-10,2021-04-01,NaN,4,NaN,it,120000.0,4.0
14,NaN,it,Analyst,150.0,₹75000,2019-05-10,2021-04-01,-2,2,yes,it,NaN,2.0
61,NaN,Finance,Senior Analyst,-5.0,120000,2020-06-10,2021-04-01,NaN,3,NaN,finance,120000.0,3.0
79,emp_79,NaN,Manager,NaN,NaN,2019-05-10,2021-04-01,8,1,False,nan,NaN,1.0
88,NaN,HR,Manager,60.0,35000,2019-05-10,2021-04-01,-2,excellent,NaN,hr,35000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,NaN,it,ANALYST,60.0,250000,2019-05-10,2021-04-01,NaN,3,no,it,250000.0,3.0
966,NaN,it,Analyst,45.0,NaN,2020-06-10,2021-04-01,1,NaN,no,it,NaN,NaN
975,NaN,NaN,Senior Analyst,28.0,55000,2020-06-10,2021-04-01,-2,NaN,yes,nan,55000.0,NaN
980,EMP1980,it,Manager,45.0,NaN,2019-05-10,2021-04-01,-2,excellent,NaN,it,NaN,NaN


In [10]:
#8. Clean experience_years and find mismatches where experience exceeds employee age. 
df['experience_years'] = pd.to_numeric(df['experience_years'], errors='coerce')
df['age'] = pd.to_numeric(df['age'], errors='coerce')

# Find mismatches: experience exceeds age
invalid_data = df[(df['experience_years'] > df['age']) & df['experience_years'].notna() & df['age'].notna()]
invalid_data

,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,dept_std,sal_std,performance_rating_num
10,emp_10,Finance,Analyst,-5.0,35000,NaT,NaT,8.0,1,no,finance,35000.0,1.0
17,emp_17,sales,Senior Analyst,-5.0,NaN,NaT,2021-04-01,3.0,4,False,sales,NaN,4.0
23,emp_23,Finance,NaN,-5.0,85000,NaT,NaT,-2.0,5,NaN,finance,85000.0,5.0
43,NaN,it,Manager,-5.0,120000,NaT,NaT,1.0,excellent,NaN,it,120000.0,NaN
49,NaN,it,mgr,-5.0,55000,2021-07-15,NaT,5.0,3,False,it,55000.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,NaN,sales,mgr,-5.0,NaN,2020-06-10,NaT,12.0,poor,no,sales,NaN,NaN
981,NaN,sales,mgr,-5.0,85000,NaT,NaT,-2.0,NaN,yes,sales,85000.0,NaN
985,emp_985,sales,mgr,-5.0,NaN,NaT,NaT,-2.0,1,NaN,sales,NaN,1.0
990,emp_990,NaN,Senior Analyst,-5.0,85000,2019-05-10,2021-04-01,12.0,5,yes,nan,85000.0,5.0


In [11]:
#9. Standardize designation and count how many active employees are in each designation. 
df['designation'] = df['designation'].astype(str).str.strip().str.lower()
df['is_active'] = df['is_active'].astype(str).str.strip().str.lower().isin(['true', 'yes', '1'])
actv_emp = df.groupby('designation')['is_active'].count().reset_index(name='actv_emp_count')
actv_emp

,designation,actv_emp_count
0,analyst,309
1,manager,158
2,mgr,178
3,nan,167
4,senior analyst,188


In [12]:
#10. Convert is_active to boolean and find inactive employees with recent promotions.

df['is_active_bool'] = (df['is_active'].astype(str).str.strip().str.lower().isin(['true', 'yes', '1']))

# Clean last_promotion_date
df['last_promotion_date_std'] = pd.to_datetime(df['last_promotion_date'],errors='coerce',dayfirst=True)

# Define recent promotion window (last 2 years)
cutoff_date = pd.Timestamp.today() - pd.DateOffset(years=2)

# Find inactive employees with recent promotions
inactive_recent_promo = df[
    (~df['is_active_bool']) &
    (df['last_promotion_date_std'].notna()) &
    (df['last_promotion_date_std'] >= cutoff_date)
]

# Display result
inactive_recent_promo[['employee_id', 'department', 'designation', 'last_promotion_date_std', 'is_active_bool']]

,employee_id,department,designation,last_promotion_date_std,is_active_bool


In [13]:
#11. Calculate employee tenure in years and find those with tenure above the 90th percentile.

# Clean joining_date
df['joining_date_std'] = pd.to_datetime(
    df['joining_date'],
    errors='coerce',
    dayfirst=True
)

# Calculate tenure in years
today = pd.Timestamp.today()

df['tenure_years'] = (
    (today - df['joining_date_std']).dt.days / 365.25
)

# Remove invalid tenure values
df['tenure_years'] = df['tenure_years'].where(df['tenure_years'] >= 0)

# Calculate 90th percentile of tenure
tenure_90th = df['tenure_years'].quantile(0.90)

# Find employees with tenure above 90th percentile
high_tenure_employees = df[
    df['tenure_years'] > tenure_90th
]

# Display result
high_tenure_employees[['employee_id', 'department', 'designation', 'joining_date_std', 'tenure_years']]

,employee_id,department,designation,joining_date_std,tenure_years


In [14]:
#12. Identify departments where more than 25% of salary values are missing or invalid. 
df['dept_std'] = df['department'].astype(str).str.strip().str.lower()

df['salary_clean'] = (
    df['salary']
    .astype(str)
    .str.replace('₹', '', regex=False)
    .str.replace(',', '', regex=False)
)

df['salary_clean'] = pd.to_numeric(df['salary_clean'], errors='coerce')

bad_departments = (
    df.groupby('dept_std')['salary_clean']
      .apply(lambda x: x.isna().mean())
      .loc[lambda x: x > 0.25]
      .index
)

invalid_valid = df[df['dept_std'].isin(bad_departments)]
invalid_valid

,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,dept_std,sal_std,performance_rating_num,is_active_bool,last_promotion_date_std,joining_date_std,tenure_years,salary_clean
2,EMP1002,Sales,nan,28.0,85000,NaT,NaT,3.0,4,False,sales,85000.0,4.0,False,NaT,NaT,NaN,85000.0
7,EMP1007,sales,analyst,22.0,NaN,2020-06-10,NaT,-2.0,1,True,sales,NaN,1.0,True,NaT,2020-06-10,5.672827,NaN
8,EMP1008,Finance,analyst,150.0,35000,2021-07-15,NaT,NaN,excellent,True,finance,35000.0,NaN,True,NaT,2021-07-15,4.577687,35000.0
10,emp_10,Finance,analyst,-5.0,35000,NaT,NaT,8.0,1,False,finance,35000.0,1.0,False,NaT,NaT,NaN,35000.0
11,NaN,sales,mgr,22.0,250000,2021-07-15,NaT,NaN,5,True,sales,250000.0,5.0,True,NaT,2021-07-15,4.577687,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,EMP1993,sales,nan,35.0,120000,2020-06-10,NaT,12.0,NaN,False,sales,120000.0,NaN,False,NaT,2020-06-10,5.672827,120000.0
994,EMP1994,Sales,nan,-5.0,85000,2019-05-10,NaT,12.0,1,True,sales,85000.0,1.0,True,NaT,2019-05-10,6.759754,85000.0
995,EMP1995,Finance,nan,60.0,55000,2019-05-10,NaT,1.0,4,True,finance,55000.0,4.0,True,NaT,2019-05-10,6.759754,55000.0
997,EMP1997,Sales,senior analyst,150.0,250000,2021-07-15,NaT,3.0,excellent,True,sales,250000.0,NaN,True,NaT,2021-07-15,4.577687,250000.0


In [15]:
#13. Create a flag for employees with high performance (≥4) but below-median salary.
# Convert performance rating to numeric
df['performance_rating_num'] = pd.to_numeric(
    df['performance_rating'], errors='coerce'
)

# Clean salary
df['salary_clean'] = (
    df['salary']
    .astype(str)
    .str.replace('₹', '', regex=False)
    .str.replace(',', '', regex=False)
)
df['salary_clean'] = pd.to_numeric(df['salary_clean'], errors='coerce')

# Calculate median salary
median_salary = df['salary_clean'].median()

# Create flag
df['high_perf_low_salary_flag'] = (
    (df['performance_rating_num'] >= 4) &
    (df['salary_clean'] < median_salary)
)

df[['employee_id', 'performance_rating_num', 'salary_clean', 'high_perf_low_salary_flag']]

,employee_id,performance_rating_num,salary_clean,high_perf_low_salary_flag
0,EMP1000,3.0,85000.0,False
1,emp_1,3.0,55000.0,False
2,EMP1002,4.0,85000.0,False
3,EMP1003,3.0,35000.0,False
4,NaN,2.0,85000.0,False
...,...,...,...,...
995,EMP1995,4.0,55000.0,True
996,EMP1996,5.0,250000.0,False
997,EMP1997,NaN,250000.0,False
998,emp_998,NaN,85000.0,False


In [16]:
#14. Detect employees with no promotion date but more than 5 years of experience. 
# Clean experience
df['experience_years'] = pd.to_numeric(
    df['experience_years'], errors='coerce'
)

# Clean promotion date
df['last_promotion_date_std'] = pd.to_datetime(
    df['last_promotion_date'], errors='coerce', dayfirst=True
)

# Filter employees
no_promo_high_exp = df[
    df['last_promotion_date_std'].isna() &
    (df['experience_years'] > 5)
]

no_promo_high_exp[
    ['employee_id', 'department', 'experience_years', 'last_promotion_date_std']
]

,employee_id,department,experience_years,last_promotion_date_std
4,NaN,IT,12.0,NaT
5,NaN,IT,8.0,NaT
6,NaN,HR,12.0,NaT
10,emp_10,Finance,8.0,NaT
12,emp_12,Sales,8.0,NaT
...,...,...,...,...
977,emp_977,IT,8.0,NaT
978,EMP1978,HR,8.0,NaT
993,EMP1993,sales,12.0,NaT
994,EMP1994,Sales,12.0,NaT


In [17]:
#15. Build a validation rule to flag rows violating at least two business constraints. 
# Clean required columns
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df['experience_years'] = pd.to_numeric(df['experience_years'], errors='coerce')
df['salary_clean'] = (
    df['salary']
    .astype(str)
    .str.replace('₹', '', regex=False)
    .str.replace(',', '', regex=False)
)
df['salary_clean'] = pd.to_numeric(df['salary_clean'], errors='coerce')

df['is_active_bool'] = (
    df['is_active']
    .astype(str)
    .str.lower()
    .isin(['true', 'yes', '1'])
)

df['last_promotion_date_std'] = pd.to_datetime(
    df['last_promotion_date'], errors='coerce', dayfirst=True
)

# Business rules
df['rule_salary_invalid'] = df['salary_clean'].isna()

df['rule_experience_invalid'] = (
    (df['experience_years'] < 0) |
    (df['experience_years'] > df['age'])
)

df['rule_age_invalid'] = (
    (df['age'] < 18) | (df['age'] > 100)
)

recent_cutoff = pd.Timestamp.today() - pd.DateOffset(years=2)
df['rule_inactive_recent_promo'] = (
    (~df['is_active_bool']) &
    (df['last_promotion_date_std'] >= recent_cutoff)
)

# Count violations
df['violation_count'] = df[
    [
        'rule_salary_invalid',
        'rule_experience_invalid',
        'rule_age_invalid',
        'rule_inactive_recent_promo'
    ]
].sum(axis=1)

# Final flag
df['validation_flag'] = df['violation_count'] >= 2

df[
    ['employee_id', 'violation_count', 'validation_flag']
]

,employee_id,violation_count,validation_flag
0,EMP1000,0,False
1,emp_1,0,False
2,EMP1002,0,False
3,EMP1003,0,False
4,NaN,1,False
...,...,...,...
995,EMP1995,0,False
996,EMP1996,1,False
997,EMP1997,1,False
998,emp_998,0,False


In [18]:
# 16  After cleaning dates, find employees whose promotion gap (promotion date − joining date) is less than 1 year. 
df['last_promotion_date_std'] = pd.to_datetime(df['last_promotion_date'], errors='coerce', dayfirst=True)

df['joining_date_std'] = pd.to_datetime(df['joining_date'],errors='coerce',dayfirst=True)

promotion_gap = df[(df['last_promotion_date_std'] - df['joining_date_std']).dt.days/365.25 < 1]
promotion_gap

,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,...,joining_date_std,tenure_years,salary_clean,high_perf_low_salary_flag,rule_salary_invalid,rule_experience_invalid,rule_age_invalid,rule_inactive_recent_promo,violation_count,validation_flag
9,EMP1009,it,nan,22.0,120000,2020-06-10,2021-04-01,NaN,4,False,...,2020-06-10,5.672827,120000.0,False,False,False,False,False,0,False
24,EMP1024,Finance,mgr,35.0,85000,2021-07-15,2021-04-01,8.0,3,True,...,2021-07-15,4.577687,85000.0,False,False,False,False,False,0,False
40,emp_40,NaN,nan,60.0,85000,2021-07-15,2021-04-01,12.0,5,True,...,2021-07-15,4.577687,85000.0,False,False,False,False,False,0,False
44,emp_44,sales,mgr,45.0,₹75000,2021-07-15,2021-04-01,8.0,2,True,...,2021-07-15,4.577687,75000.0,False,False,False,False,False,0,False
47,NaN,Finance,analyst,NaN,250000,2021-07-15,2021-04-01,3.0,excellent,True,...,2021-07-15,4.577687,250000.0,False,False,False,False,False,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,EMP1943,Finance,mgr,45.0,35000,2020-06-10,2021-04-01,-2.0,NaN,False,...,2020-06-10,5.672827,35000.0,False,False,True,False,False,1,False
945,NaN,HR,manager,-5.0,35000,2020-06-10,2021-04-01,3.0,NaN,True,...,2020-06-10,5.672827,35000.0,False,False,True,True,False,2,True
966,NaN,it,analyst,45.0,NaN,2020-06-10,2021-04-01,1.0,NaN,False,...,2020-06-10,5.672827,NaN,False,True,False,False,False,1,False
974,emp_974,NaN,mgr,NaN,85000,2021-07-15,2021-04-01,3.0,4,False,...,2021-07-15,4.577687,85000.0,False,False,False,False,False,0,False


In [19]:
#17 Identify employees whose salary is above the department average but performance rating is below the department median. 
df['sal_std'] = (df['salary'].astype(str).str.replace('₹', '', regex=False).str.replace(',', '', regex=False))
df['sal_std'] = pd.to_numeric(df['sal_std'], errors='coerce')
df['dept_std'] = df['department'].astype(str).str.strip().str.lower()
df['performance_rating_num'] = pd.to_numeric(df['performance_rating'], errors='coerce')
df['dept_avg_salary'] = (df.groupby('dept_std')['sal_std'].transform('mean'))
df['dept_median_perf'] = (df.groupby('dept_std')['performance_rating_num'].transform('median'))
result = df[(df['sal_std'] > df['dept_avg_salary']) & (df['performance_rating_num'] < df['dept_median_perf'])]
result

,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,...,salary_clean,high_perf_low_salary_flag,rule_salary_invalid,rule_experience_invalid,rule_age_invalid,rule_inactive_recent_promo,violation_count,validation_flag,dept_avg_salary,dept_median_perf
45,NaN,Finance,nan,NaN,120000,NaT,2021-04-01,1.0,1,True,...,120000.0,False,False,False,False,False,0,False,107397.959184,3.0
89,emp_89,NaN,senior analyst,22.0,120000,2020-06-10,NaT,1.0,1,False,...,120000.0,False,False,False,False,False,0,False,96039.603960,3.0
115,emp_115,sales,analyst,28.0,250000,2021-07-15,NaT,-2.0,2,False,...,250000.0,False,False,True,False,False,1,False,110842.696629,3.0
127,NaN,Finance,nan,150.0,120000,NaT,NaT,3.0,1,True,...,120000.0,False,False,False,True,False,1,False,107397.959184,3.0
157,NaN,Finance,analyst,22.0,250000,2019-05-10,NaT,3.0,1,False,...,250000.0,False,False,False,False,False,0,False,107397.959184,3.0
169,emp_169,it,mgr,60.0,120000,2021-07-15,2021-04-01,NaN,2,False,...,120000.0,False,False,False,False,False,0,False,103373.205742,3.0
173,emp_173,Finance,manager,45.0,120000,2020-06-10,NaT,8.0,1,False,...,120000.0,False,False,False,False,False,0,False,107397.959184,3.0
216,emp_216,NaN,senior analyst,60.0,120000,NaT,NaT,8.0,2,False,...,120000.0,False,False,False,False,False,0,False,96039.603960,3.0
217,NaN,it,nan,28.0,250000,2021-07-15,NaT,NaN,2,True,...,250000.0,False,False,False,False,False,0,False,103373.205742,3.0
223,NaN,HR,manager,NaN,120000,2019-05-10,NaT,-2.0,1,True,...,120000.0,False,False,True,False,False,1,False,110117.647059,3.0


In [20]:
#18 Create a column that categorizes employees into age bands (Young, Mid, Senior) and count employees per band per department.
df['age_num'] = pd.to_numeric(df['age'], errors='coerce')

df['age_band'] = pd.cut(
    df['age_num'],
    bins=[0, 29, 45, np.inf],
    labels=['Young', 'Mid', 'Senior']
)

df['dept_std'] = df['department'].astype(str).str.strip().str.lower()

age_band_count = (
    df
    .groupby(['dept_std', 'age_band'], observed=True)
    .size()
    .reset_index(name='employee_count')
)

age_band_count

,dept_std,age_band,employee_count
0,finance,Young,33
1,finance,Mid,31
2,finance,Senior,24
3,hr,Young,54
4,hr,Mid,52
5,hr,Senior,36
6,it,Young,53
7,it,Mid,78
8,it,Senior,58
9,nan,Young,26


In [21]:
#19 Find departments where the average experience is higher than the company-wide average experience.

df['experience_num'] = pd.to_numeric(df['experience_years'], errors='coerce')

# Standardize department names
df['dept_std'] = df['department'].astype(str).str.strip().str.lower()

# Company-wide average experience
company_avg_experience = df['experience_num'].mean()

# Department-wise average experience
dept_avg_experience = (df.groupby('dept_std', observed=True)['experience_num'].mean().reset_index(name='dept_avg_experience'))

# Filter departments above company average
result = dept_avg_experience[dept_avg_experience['dept_avg_experience'] > company_avg_experience]

# Display result
result

,dept_std,dept_avg_experience
0,finance,4.826923
4,sales,4.709677


In [23]:
#20 Detect employees marked as inactive but having a valid promotion date within the last 2 years. 
# Clean is_active to boolean
df['is_active_bool'] = (
    df['is_active']
    .astype(str)
    .str.strip()
    .str.lower()
    .isin(['true', 'yes', '1'])
)

# Clean promotion date
df['last_promotion_date_std'] = pd.to_datetime(
    df['last_promotion_date'], errors='coerce', dayfirst=True
)

# Define cutoff (last 2 years)
cutoff_date = pd.Timestamp.today() - pd.DateOffset(years=2)

# Filter inactive but recently promoted employees
inactive_recent_promo = df[
    (~df['is_active_bool']) &
    (df['last_promotion_date_std'].notna()) &
    (df['last_promotion_date_std'] >= cutoff_date)
]

inactive_recent_promo

,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,...,rule_experience_invalid,rule_age_invalid,rule_inactive_recent_promo,violation_count,validation_flag,dept_avg_salary,dept_median_perf,age_num,age_band,experience_num


In [24]:
#21 Identify designations where more than 20% of employees have missing or invalid experience values. 
# Clean experience
df['experience_num'] = pd.to_numeric(df['experience_years'], errors='coerce')

# Standardize designation
df['designation_std'] = df['designation'].astype(str).str.strip().str.lower()

# Calculate invalid experience percentage per designation
designation_exp_issue = (
    df
    .assign(invalid_exp=lambda x: x['experience_num'].isna())
    .groupby('designation_std', observed=True)
    .agg(
        total_employees=('experience_num', 'size'),
        invalid_experience=('invalid_exp', 'sum')
    )
    .assign(
        invalid_pct=lambda x: x['invalid_experience'] / x['total_employees']
    )
    .reset_index()
)

# Filter > 20%
designation_exp_issue[designation_exp_issue['invalid_pct'] > 0.20]

,designation_std,total_employees,invalid_experience,invalid_pct
2,mgr,178,47,0.264045
4,senior analyst,188,38,0.202128


In [25]:
#22 Calculate the ratio of salary to experience years and detect extreme values using the 95th percentile. 
# Clean salary
df['salary_num'] = (
    df['salary']
    .astype(str)
    .str.replace('₹', '', regex=False)
    .str.replace(',', '', regex=False)
)
df['salary_num'] = pd.to_numeric(df['salary_num'], errors='coerce')

# Salary to experience ratio
df['sal_exp_ratio'] = df['salary_num'] / df['experience_num']

# 95th percentile threshold
ratio_95 = df['sal_exp_ratio'].quantile(0.95)

# Extreme values
extreme_ratio_employees = df[df['sal_exp_ratio'] > ratio_95]

extreme_ratio_employees

,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,...,violation_count,validation_flag,dept_avg_salary,dept_median_perf,age_num,age_band,experience_num,designation_std,salary_num,sal_exp_ratio
124,NaN,sales,analyst,NaN,250000,2021-07-15,2021-04-01,1.0,5,False,...,0,False,110842.696629,3.0,NaN,NaN,1.0,analyst,250000.0,250000.0
133,NaN,it,nan,-5.0,250000,2021-07-15,NaT,1.0,5,True,...,2,True,103373.205742,3.0,-5.0,NaN,1.0,nan,250000.0,250000.0
176,NaN,it,analyst,22.0,250000,NaT,NaT,1.0,NaN,False,...,0,False,103373.205742,3.0,22.0,Young,1.0,analyst,250000.0,250000.0
224,NaN,Finance,manager,60.0,250000,2021-07-15,NaT,1.0,3,False,...,0,False,107397.959184,3.0,60.0,Senior,1.0,manager,250000.0,250000.0
262,NaN,IT,manager,-5.0,250000,NaT,NaT,1.0,5,False,...,2,True,103373.205742,3.0,-5.0,NaN,1.0,manager,250000.0,250000.0
399,NaN,Finance,analyst,60.0,250000,NaT,NaT,1.0,3,True,...,0,False,107397.959184,3.0,60.0,Senior,1.0,analyst,250000.0,250000.0
401,EMP1401,Finance,analyst,22.0,250000,NaT,NaT,1.0,1,False,...,0,False,107397.959184,3.0,22.0,Young,1.0,analyst,250000.0,250000.0
470,EMP1470,HR,mgr,35.0,250000,2021-07-15,NaT,1.0,excellent,True,...,0,False,110117.647059,3.0,35.0,Mid,1.0,mgr,250000.0,250000.0
525,emp_525,sales,manager,-5.0,250000,2020-06-10,NaT,1.0,1,False,...,2,True,110842.696629,3.0,-5.0,NaN,1.0,manager,250000.0,250000.0
529,emp_529,NaN,nan,28.0,250000,2020-06-10,NaT,1.0,5,False,...,0,False,96039.603960,3.0,28.0,Young,1.0,nan,250000.0,250000.0


In [26]:
#23 Find employees whose age is valid but inconsistent with experience (experience > age − 18). 
# Clean age
df['age_num'] = pd.to_numeric(df['age'], errors='coerce')

# Inconsistent rows
age_exp_inconsistent = df[
    df['age_num'].notna() &
    df['experience_num'].notna() &
    (df['experience_num'] > (df['age_num'] - 18))
]

age_exp_inconsistent

,employee_id,department,designation,age,salary,joining_date,last_promotion_date,experience_years,performance_rating,is_active,...,violation_count,validation_flag,dept_avg_salary,dept_median_perf,age_num,age_band,experience_num,designation_std,salary_num,sal_exp_ratio
10,emp_10,Finance,analyst,-5.0,35000,NaT,NaT,8.0,1,False,...,2,True,107397.959184,3.0,-5.0,NaN,8.0,analyst,35000.0,4375.000000
17,emp_17,sales,senior analyst,-5.0,NaN,NaT,2021-04-01,3.0,4,False,...,3,True,110842.696629,3.0,-5.0,NaN,3.0,senior analyst,NaN,NaN
23,emp_23,Finance,nan,-5.0,85000,NaT,NaT,-2.0,5,False,...,2,True,107397.959184,3.0,-5.0,NaN,-2.0,nan,85000.0,-42500.000000
43,NaN,it,manager,-5.0,120000,NaT,NaT,1.0,excellent,False,...,2,True,103373.205742,3.0,-5.0,NaN,1.0,manager,120000.0,120000.000000
49,NaN,it,mgr,-5.0,55000,2021-07-15,NaT,5.0,3,False,...,2,True,103373.205742,3.0,-5.0,NaN,5.0,mgr,55000.0,11000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,emp_968,IT,manager,22.0,120000,2021-07-15,NaT,5.0,poor,True,...,0,False,103373.205742,3.0,22.0,Young,5.0,manager,120000.0,24000.000000
981,NaN,sales,mgr,-5.0,85000,NaT,NaT,-2.0,NaN,True,...,2,True,110842.696629,3.0,-5.0,NaN,-2.0,mgr,85000.0,-42500.000000
985,emp_985,sales,mgr,-5.0,NaN,NaT,NaT,-2.0,1,False,...,3,True,110842.696629,3.0,-5.0,NaN,-2.0,mgr,NaN,NaN
990,emp_990,NaN,senior analyst,-5.0,85000,2019-05-10,2021-04-01,12.0,5,True,...,2,True,96039.603960,3.0,-5.0,NaN,12.0,senior analyst,85000.0,7083.333333


In [27]:
#24 Compute department-wise attrition rate assuming inactive employees represent attrition. 
# Standardize department
df['dept_std'] = df['department'].astype(str).str.strip().str.lower()

# Attrition calculation
dept_attrition = (
    df
    .groupby('dept_std', observed=True)
    .agg(
        total_employees=('employee_id', 'count'),
        inactive_employees=('is_active_bool', lambda x: (~x).sum())
    )
    .assign(
        attrition_rate=lambda x: x['inactive_employees'] / x['total_employees']
    )
    .reset_index()
)

dept_attrition

,dept_std,total_employees,inactive_employees,attrition_rate
0,finance,72,73,1.013889
1,hr,89,122,1.370787
2,it,150,160,1.066667
3,nan,70,78,1.114286
4,sales,115,145,1.260870


In [28]:
#25 Create a final quality score per row based on number of valid fields and flag rows below a quality threshold. 

# Clean performance rating
df['performance_rating_num'] = pd.to_numeric(
    df['performance_rating'], errors='coerce'
)

# Define valid field checks
valid_fields = {
    'age': df['age_num'].notna(),
    'experience': df['experience_num'].notna(),
    'salary': df['salary_num'].notna(),
    'department': df['department'].notna(),
    'designation': df['designation'].notna(),
    'performance_rating': df['performance_rating_num'].notna()
}

# Compute quality score
df['quality_score'] = sum(valid_fields.values())

# Define threshold (example: at least 4 valid fields)
quality_threshold = 4

# Flag low-quality rows
df['low_quality_flag'] = df['quality_score'] < quality_threshold

df[['employee_id', 'quality_score', 'low_quality_flag']]

,employee_id,quality_score,low_quality_flag
0,EMP1000,6,False
1,emp_1,5,False
2,EMP1002,6,False
3,EMP1003,6,False
4,NaN,6,False
...,...,...,...
995,EMP1995,6,False
996,EMP1996,6,False
997,EMP1997,5,False
998,emp_998,5,False
